In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from google.colab import drive

#  Monter Google Drive
drive.mount('/content/drive')

#  Chemins
drive_base_path = "/content/drive/MyDrive/Colab Notebooks/fahim/final"
local_base_path = "/content/modeles_importes"
os.makedirs(local_base_path, exist_ok=True)

#  1. Modèles à copier (images + textuels)
model_folders = {
    "modeles_image_v2": ["efficientv2L_best.pt", "convnextv2_best.pt", "coatnet2_best.pt","Maxvit_best.pt", "gcvit_best.pt"],
    "camembert2_model": ["config.json", "model.safetensors"],
    "flaubert2_model": ["config.json", "model.safetensors"],
}

# 📥Copie des modèles
for folder, files in model_folders.items():
    src_dir = os.path.join(drive_base_path, folder)
    is_text_model = "camembert" in folder or "flaubert" in folder
    dst_subfolder = os.path.join(local_base_path, os.path.basename(folder)) if is_text_model else local_base_path
    os.makedirs(dst_subfolder, exist_ok=True)

    for filename in files:
        src_file = os.path.join(src_dir, filename)
        dst_file = os.path.join(dst_subfolder, filename)
        if os.path.exists(src_file):
            shutil.copy(src_file, dst_file)
            print(f"✅ Copié : {filename} → {dst_file}")
        else:
            print(f"❌ Introuvable : {src_file}")

#  2. Logits à copier (texte + image + labels)
logits_files = [
    "efficientv2L_logits.pt",
    "convnextv2_logits.pt",
    "coatnet2_logits.pt",
    "Maxvit_logits.pt",
    "gcvit_logits.pt",
    "camembert2_logits.pt",
    "flaubert2_logits.pt",
    "true_labels_final.pt"
]

logits_drive_path = drive_base_path

for filename in logits_files:
    src_file = os.path.join(logits_drive_path, filename)
    dst_file = os.path.join(local_base_path, filename)
    if os.path.exists(src_file):
        shutil.copy(src_file, dst_file)
        print(f"✅ Logits copié : {filename}")
    else:
        print(f"❌ Logits manquant : {src_file}")
# 3. Copier le fichier de mapping labels
mapping_filename = "label_mapping_final.json"
mapping_src = os.path.join(drive_base_path, mapping_filename)
mapping_dst = os.path.join(local_base_path, mapping_filename)

if os.path.exists(mapping_src):
    shutil.copy(mapping_src, mapping_dst)
    print(f"✅ Mapping copié : {mapping_filename}")
else:
    print(f"❌ Mapping manquant : {mapping_src}")
# 4. Copier le fichier label encoder
label_encoder_filename = "label_encoder_final.pkl"
label_encoder_src = os.path.join(drive_base_path, label_encoder_filename)
label_encoder_dst = os.path.join(local_base_path, label_encoder_filename)

if os.path.exists(label_encoder_src):
    shutil.copy(label_encoder_src, label_encoder_dst)
    print(f"✅ Label Encoder copié : {label_encoder_filename}")
else:
    print(f"❌ Label Encoder manquant : {label_encoder_src}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Copié : efficientv2L_best.pt → /content/modeles_importes/efficientv2L_best.pt
✅ Copié : convnextv2_best.pt → /content/modeles_importes/convnextv2_best.pt
✅ Copié : coatnet2_best.pt → /content/modeles_importes/coatnet2_best.pt
✅ Copié : Maxvit_best.pt → /content/modeles_importes/Maxvit_best.pt
✅ Copié : gcvit_best.pt → /content/modeles_importes/gcvit_best.pt
✅ Copié : config.json → /content/modeles_importes/camembert2_model/config.json
✅ Copié : model.safetensors → /content/modeles_importes/camembert2_model/model.safetensors
✅ Copié : config.json → /content/modeles_importes/flaubert2_model/config.json
✅ Copié : model.safetensors → /content/modeles_importes/flaubert2_model/model.safetensors
✅ Logits copié : efficientv2L_logits.pt
✅ Logits copié : convnextv2_logits.pt
✅ Logits copié : coatnet2_logits.pt
✅ Logits copié : Maxvit_logits.pt
✅ Logits copié : gcvit_

In [ ]:
import os
import torch
import numpy as np
import itertools
from sklearn.metrics import f1_score
import json

# Chemins et config
PATH_LOGITS = "/content/modeles_importes"
LABELS_PATH = os.path.join(PATH_LOGITS, "true_labels_final.pt")
VAL_INDICES_PATH = "/content/drive/MyDrive/Colab Notebooks/fahim/final/val_indices.json"

# Liste des modèles à tester (mets tous les noms exacts des logits, sans "_logits.pt")
all_model_names = [
    "camembert2", "flaubert2", "gcvit", "convnextv2", "Maxvit",
    "coatnet2", "efficientv2L" # Ajoute/retire ici
]

# Chargement logits
all_logits = {name: torch.load(os.path.join(PATH_LOGITS, f"{name}_logits.pt")).cpu().numpy() for name in all_model_names}
labels = torch.load(LABELS_PATH).numpy()
with open(VAL_INDICES_PATH, "r") as f:
    val_idx = np.array(json.load(f), dtype=int)
y_val = labels[val_idx]

# Fonction pour pondérer
def weighted_softmax(logits_list, weights):
    """Calcule la moyenne pondérée des logits softmaxés."""
    softmaxed = [np.exp(logits)/np.exp(logits).sum(axis=1, keepdims=True) for logits in logits_list]
    weighted = sum(w * s for w, s in zip(weights, softmaxed))
    return weighted

# Grid-search sur pondération (pas de 0.1, tu peux affiner)
def grid_weights(n):
    """Génère toutes les combinaisons de poids qui font 1 (step=0.1)."""
    steps = np.arange(0.1, 1.0, 0.1)
    grids = [x for x in itertools.product(steps, repeat=n) if abs(sum(x) - 1) < 1e-5]
    return grids

# Test de toutes les combinaisons de modèles
results = []
for n_models in range(2, len(all_model_names)+1):
    for model_combo in itertools.combinations(all_model_names, n_models):
        logits_list = [all_logits[name][val_idx] for name in model_combo]
        best_f1, best_weights = 0, None
        for weights in grid_weights(n_models):
            y_pred = np.argmax(weighted_softmax(logits_list, weights), axis=1)
            f1 = f1_score(y_val, y_pred, average='weighted')
            if f1 > best_f1:
                best_f1 = f1
                best_weights = weights
        results.append({
            "models": model_combo,
            "f1_weighted": best_f1,
            "weights": best_weights
        })
        print(f"[{'+'.join(model_combo)}]  F1_weighted={best_f1:.5f}  Weights={best_weights}")

# Affichage trié
results = sorted(results, key=lambda x: x["f1_weighted"], reverse=True)
print("\n==== TOP COMBINAISONS ====")
for res in results[:10]:
    print(f"{'+'.join(res['models']):40s} | F1={res['f1_weighted']:.5f} | Weights={res['weights']}")


<ipython-input-30-01b3eb855353>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_logits = {name: torch.load(os.path.join(PATH_LOGITS, f"{name}_logits.pt")).cpu().numpy(

[camembert2+flaubert2]  F1_weighted=0.89551  Weights=(0.5, 0.5)
[camembert2+gcvit]  F1_weighted=0.89785  Weights=(0.5, 0.5)
[camembert2+convnextv2]  F1_weighted=0.89901  Weights=(0.5, 0.5)
[camembert2+Maxvit]  F1_weighted=0.89931  Weights=(0.5, 0.5)
[camembert2+coatnet2]  F1_weighted=0.89452  Weights=(0.5, 0.5)
[camembert2+efficientv2L]  F1_weighted=0.89803  Weights=(0.5, 0.5)
[flaubert2+gcvit]  F1_weighted=0.90065  Weights=(0.5, 0.5)
[flaubert2+convnextv2]  F1_weighted=0.89915  Weights=(0.5, 0.5)
[flaubert2+Maxvit]  F1_weighted=0.89949  Weights=(0.5, 0.5)
[flaubert2+coatnet2]  F1_weighted=0.89762  Weights=(0.5, 0.5)
[flaubert2+efficientv2L]  F1_weighted=0.90023  Weights=(0.5, 0.5)
[gcvit+convnextv2]  F1_weighted=0.78196  Weights=(0.5, 0.5)
[gcvit+Maxvit]  F1_weighted=0.78286  Weights=(0.5, 0.5)
[gcvit+coatnet2]  F1_weighted=0.77597  Weights=(0.6, 0.4)
[gcvit+efficientv2L]  F1_weighted=0.77920  Weights=(0.5, 0.5)
[convnextv2+Maxvit]  F1_weighted=0.78255  Weights=(0.5, 0.5)
[convnextv2+

In [ ]:
import numpy as np
import torch

# Weights de la combinaison gagnante
weights = np.array([0.3, 0.2, 0.1, 0.2, 0.1, 0.1]) # Issu de la cellule precedente

# Charge logits de tous les modèles, full dataset
logits_list = [
    torch.load("/content/modeles_importes/camembert2_logits.pt").cpu().numpy(),
    torch.load("/content/modeles_importes/flaubert2_logits.pt").cpu().numpy(),
    torch.load("/content/modeles_importes/convnextv2_logits.pt").cpu().numpy(),
    torch.load("/content/modeles_importes/Maxvit_logits.pt").cpu().numpy(),
    torch.load("/content/modeles_importes/coatnet2_logits.pt").cpu().numpy(),
    torch.load("/content/modeles_importes/efficientv2L_logits.pt").cpu().numpy(),
]

# Pondération → shape finale : (N, nb_classes)
stacked = np.stack(logits_list, axis=0)  # (6, N, nb_classes)
weighted_logits = np.tensordot(weights, stacked, axes=([0], [0]))  # (N, nb_classes)
weighted_probs = torch.softmax(torch.tensor(weighted_logits), dim=1).numpy()

# Sauvegarde sur tout le dataset
torch.save(torch.tensor(weighted_probs), "/content/drive/MyDrive/Colab Notebooks/fahim/final/booster_logits.pt")
print("Booster logits pondérés générés sur tout le dataset !")


Booster logits pondérés générés sur tout le dataset !
